In [8]:
from pyiceberg.catalog import NessieCatalog
from pyiceberg.schema import Schema, NestedField
from pyiceberg.types import StringType, IntegerType, DoubleType
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import boto3
from io import BytesIO
from pyiceberg.exceptions import NamespaceAlreadyExistsError, TableAlreadyExistsError

# Define the warehouse path and catalog configuration
warehouse_path = "s3://warehouse"
catalog = NessieCatalog(
    "exampledb",
    **{
        "uri": "http://localhost:19120/api/v2",  # Nessie Server URI
        "warehouse": warehouse_path,
        "s3.endpoint": "http://localhost:9000",
        "s3.region": "us-east-1",
        "s3.access-key-id": "admin",
        "s3.secret-access-key": "password",
    },
)

ImportError: cannot import name 'NessieCatalog' from 'pyiceberg.catalog' (c:\Projects\WashU\myenv\Lib\site-packages\pyiceberg\catalog\__init__.py)

In [ ]:
p
    print(f"Namespace '{namespace}' created successfully.")
except NamespaceAlreadyExistsError:
    print(f"Namespace '{namespace}' already exists.")

# Define the schema for the table using NestedField
schema = Schema(
    NestedField(field_id=1, name="id", field_type=IntegerType(), required=True),
    NestedField(field_id=2, name="name", field_type=StringType(), required=True),
    NestedField(field_id=3, name="value", field_type=DoubleType(), required=True)
)

In [ ]:



# Check if the table exists, and drop it if it does
table_name = f"{namespace}.messages"
if table_name in catalog.list_tables(namespace):
    catalog.drop_table(table_name)
    print(f"Table '{table_name}' dropped successfully.")

# Create the table
try:
    table = catalog.create_table(table_name, schema=schema)
    print(f"Table '{table_name}' created successfully.")
except TableAlreadyExistsError:
    print(f"Table '{table_name}' already exists and could not be dropped.")

# Create a sample DataFrame
data = {
    "id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "value": [123.45, 678.90, 234.56]
}
df = pd.DataFrame(data)


In [ ]:


# Convert the DataFrame to a Parquet file in memory
parquet_buffer = BytesIO()
df.to_parquet(parquet_buffer, index=False)
parquet_buffer.seek(0)

# Initialize the S3 client
s3_client = boto3.client(
    "s3",
    aws_access_key_id="admin",
    aws_secret_access_key="password",
    endpoint_url="http://localhost:9000",
    region_name="us-east-1"
)

# Upload the Parquet file to S3
s3_client.put_object(Bucket="warehouse", Key=f"{namespace}/messages/data.parquet", Body=parquet_buffer.getvalue())
print("Data uploaded to S3 successfully.")

# Verify metadata in Dremio
import requests

dremio_url = "http://localhost:9047/api/v3/catalog"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Basic " + "YOUR_BASE64_ENCODED_CREDENTIALS"
}

response = requests.get(dremio_url, headers=headers)
if response.status_code == 200:
    print("Metadata for 'demosaturday.messages':", response.json())
else:
    print("Failed to retrieve metadata from Dremio:", response.text)